Загрузим бандл

In [14]:
project_name = 'gg_project'
dataset_name = 'gg_dataset'
bucket = 'ggbucket'


In [15]:
from pathlib import Path
from tg.grammar_ru.common.loc import Loc
from dotenv import load_dotenv
load_dotenv(Loc.root_path / 'environment.env')


True

Создадим бакет

In [16]:
from tg.grammar_ru.ml.components.yandex_storage.s3_yandex_helpers import S3YandexHandler
# S3YandexHandler.create_bucket(bucket)

In [17]:
bundle_folder = Loc.data_cache_path/'bundles/grammatical_gender/toy'

In [18]:
from tg.common import DataBundle
from tg.common.ml.batched_training import IndexedDataBundle

db = DataBundle.load(bundle_folder)
idb = IndexedDataBundle(db.index, db)

In [19]:
s= db.src

In [20]:
s3path = f'datasphere/{project_name}/datasets/{dataset_name}'
# S3YandexHandler.upload_folder(bucket, s3path, bundle_folder)


In [21]:
from tg.common._common import Loc
from tg.grammar_ru.ml.components.yandex_storage.s3_yandex_helpers import S3YandexHandler

Далее - `gg_run_training.py`

In [22]:
from tg.common._common import Loc
from tg.grammar_ru.ml.components.yandex_storage.s3_yandex_helpers import S3YandexHandler
from tg.grammar_ru.ml.components.yandex_delivery.datasphere_tools import *
import ast
from tg.grammar_ru.ml.components.yandex_delivery.training_logs import S3TrainingLogsLoader, TrainingLogsViewer
from typing import List


In [23]:
tasks = get_tasks_list('datasphere/gg_project/job_info/job_gg_project_12:33:43.816522.txt',
                       bucket)

loader = S3TrainingLogsLoader(bucket, project_name)
metrics = loader.load_metrics(tasks)
metrics

output/
output/batcher.pkl
output/history.pkl
output/info.pkl
output/metrics.pkl
output/model.pkl
output/result_df.parquet
output/test_splits.pkl
output/train_split.pkl
output/training_task.pkl


,metric,ordinal,value,timestamp,job_id
0,accuracy_test,0,0.385852,2022-12-13 12:35:29.618700,gg_task 12:33:43.816673
1,accuracy_display,0,0.380557,2022-12-13 12:35:29.618700,gg_task 12:33:43.816673
2,loss,0,0.190825,2022-12-13 12:35:29.618700,gg_task 12:33:43.816673
3,iteration,0,0.000000,2022-12-13 12:35:29.618700,gg_task 12:33:43.816673
4,accuracy_test,1,0.385852,2022-12-13 12:37:07.374937,gg_task 12:33:43.816673
5,accuracy_display,1,0.380557,2022-12-13 12:37:07.374937,gg_task 12:33:43.816673
6,loss,1,0.180427,2022-12-13 12:37:07.374937,gg_task 12:33:43.816673
7,iteration,1,1.000000,2022-12-13 12:37:07.374937,gg_task 12:33:43.816673


In [24]:
tasks = get_tasks_list('datasphere/gg_project/job_info/job_gg_project_11:38:18.460849.txt',
                       bucket)

loader = S3TrainingLogsLoader(bucket, project_name)
metrics = loader.load_metrics(tasks)
metrics

output/
output/batcher.pkl
output/history.pkl
output/info.pkl
output/metrics.pkl
output/model.pkl
output/result_df.parquet
output/test_splits.pkl
output/train_split.pkl
output/training_task.pkl


,metric,ordinal,value,timestamp,job_id
0,accuracy_test,0,0.385852,2022-12-09 11:39:47.973667,gg_task 11:38:18.460985
1,accuracy_display,0,0.380557,2022-12-09 11:39:47.973667,gg_task 11:38:18.460985
2,loss,0,0.185628,2022-12-09 11:39:47.973667,gg_task 11:38:18.460985
3,iteration,0,0.000000,2022-12-09 11:39:47.973667,gg_task 11:38:18.460985
4,accuracy_test,1,0.385852,2022-12-09 11:41:16.658111,gg_task 11:38:18.460985
5,accuracy_display,1,0.380557,2022-12-09 11:41:16.658111,gg_task 11:38:18.460985
6,loss,1,0.183456,2022-12-09 11:41:16.658111,gg_task 11:38:18.460985
7,iteration,1,1.000000,2022-12-09 11:41:16.658111,gg_task 11:38:18.460985


In [25]:
TrainingLogsViewer.get_metric_by_job(metrics, 'accuracy_display')

job_id,gg_task 11:38:18.460985
0,0.380557
1,0.380557


In [26]:
import matplotlib.pyplot as plt
# plt.plot(TrainingLogsViewer.get_metric_by_job(metrics, 'accuracy_display'))

2022-12-12 07:55:05.076267+00:00 INFO: ###METRIC###accuracy_test:0.3858517058015176###
2022-12-12 07:55:05.076363+00:00 INFO: ###METRIC###accuracy_display:0.38055688336520077###
2022-12-12 07:55:05.076411+00:00 INFO: ###METRIC###loss:0.17997605365995561###
2022-12-12 07:55:05.076453+00:00 INFO: ###METRIC###iteration:19###

2022-12-12 07:53:38.507928+00:00 INFO: ###METRIC###accuracy_test:0.3858517058015176###
2022-12-12 07:53:38.508022+00:00 INFO: ###METRIC###accuracy_display:0.38055688336520077###
2022-12-12 07:53:38.508074+00:00 INFO: ###METRIC###loss:0.18003776825954912###
2022-12-12 07:53:38.508115+00:00 INFO: ###METRIC###iteration:18###

'__class__', '__delattr__', '__dir__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_check_training_time_conditions', '_ensure_bundle', '_evaluation_df', '_evaluation_for_one_stage', '_get_split', '_instantiate_all', '_prepare_all', '_train_epoch_with_minibatches', '_train_simple_epoch', '_training_report', '_wait_till_end_of_quite_hours', 'apply_hyperparams', 'generate_sample_batch', 'get_metric_names', 'predict', 'run', 'run_with_environment', 'splitter'

In [32]:
net

FeedForwardNetwork(
  (networks): ModuleList(
    (0): FeedForwardNetwork(
      (networks): ModuleList(
        (0): ExtractingNetwork()
        (1): FullyConnectedNetwork(
          (layers): ModuleList(
            (0): Linear(in_features=466, out_features=30, bias=True)
          )
        )
      )
    )
    (1): FullyConnectedNetwork(
      (layers): ModuleList(
        (0): Linear(in_features=30, out_features=3, bias=True)
        (1): Linear(in_features=3, out_features=4, bias=True)
      )
    )
  )
)

In [33]:
from tg.common import DataBundle
from tg.common.ml.batched_training import IndexedDataBundle

from tg.common.ml import batched_training as bt
from tg.common.ml.batched_training import torch as btt
from tg.common.ml.batched_training import context as btc
from tg.common.ml import dft
from yo_fluq_ds import fluq


from tg.grammar_ru.ml.components.attention_network import AttentionNetwork
from tg.common.ml.batched_training.torch.networks.lstm_network import LSTMFinalizer

from tg.grammar_ru.ml.components.extractor_settings import CoreExtractor
from tg.grammar_ru.ml.components.plain_context_builder import PlainContextBuilder
import datetime
from tg.grammar_ru.ml.components.training_task_factory import TaskFactory
from tg.grammar_ru.ml.components.training_task_factory import Conventions
from tg.common.ml.training_core import TrainingEnvironment
from tg.grammar_ru.ml.components.contextual_binding import ContextualBinding, ContextualNetworkType

from tg.grammar_ru.ml.components.yandex_delivery.training_job import TrainingJob
from tg.grammar_ru.ml.components.yandex_storage.s3_yandex_helpers import S3YandexHandler
from tg.common.delivery.jobs import SSHDockerJobRoutine, DockerOptions
from tg.common.delivery.packaging import FakeContainerHandler

from tg.grammar_ru.common import Loc
from sklearn.metrics import roc_auc_score
from typing import Dict, Optional  # TODO delete redundant
from tg.common.delivery.training.architecture import FileCacheTrainingEnvironment
from tg.grammar_ru.ml.components.yandex_delivery.docker_tools import deploy_container
from tg.common.delivery.jobs.ssh_docker_job_routine import build_container
from tg.common.ml.batched_training.torch.networks import FeedForwardNetwork, FullyConnectedNetwork
import torch
from pathlib import Path
from dotenv import load_dotenv

from tg.grammar_ru.common import Loc

load_dotenv(Loc.root_path / 'environment.env')

project_name = 'gg_project'
dataset_name = 'gg_dataset'
bucket = 'ggbucket'


class MulticlassMetrics(bt.Metric):
    def __init__(self, add_accuracy=True, add_rating=False):
        self.add_accuracy = add_accuracy
        self.add_rating = add_rating

    def get_names(self):
        result = []
        if self.add_accuracy:
            result.append('accuracy')
        if self.add_rating:
            result.append('rating')
        return result

    def measure(self, df, _):
        prefix = 'true_label_'
        labels = []
        for c in df.columns:
            if c.startswith(prefix):
                labels.append(c.replace(prefix, ''))

        def ustack(df, prefix, cols, name):
            df = df[[prefix+c for c in cols]]
            df.columns = [c for c in cols]
            df = df.unstack().to_frame(name)
            return df

        predicted = ustack(df, 'predicted_label_', labels, 'predicted')
        true = ustack(df, 'true_label_', labels, 'true')
        df = predicted.merge(true, left_index=True,
                             right_index=True).reset_index()
        df.columns = ['label', 'sample', 'predicted', 'true']
        df = df.feed(fluq.add_ordering_column(
            'sample', ('predicted', False), 'predicted_rating'))

        match = (df.loc[df.predicted_rating ==
                 0].set_index('sample').true > 0.5)
        rating = df.loc[df.true > 0.5].set_index('sample').predicted_rating
        result = []
        if self.add_accuracy:
            result.append(match.mean())
        if self.add_rating:
            result.append(rating.mean())
        return result


def get_multilabel_extractor():
    label_extractor = (bt.PlainExtractor
                       .build(Conventions.LabelFrame)
                       .index()
                       .apply(take_columns=['label'], transformer=dft.DataFrameTransformerFactory.default_factory())
                       )
    return label_extractor


class MyNetworkFactory:
    def __init__(self, nn_head_factory):
        self.nn_head_factory = nn_head_factory

    def create_network(self, task, input):  # input is batch ~ sample
        nn_head = self.nn_head_factory.create_network(task=None, input=input)
        head_out = nn_head(input)
        hidden_size = head_out.shape[1]
        labels_count = input['label'].shape[1]
        nn_tail = FullyConnectedNetwork(
            sizes=[], input=hidden_size, output=labels_count)
        # return FeedForwardNetwork(nn_head, nn_tail, torch.nn.Softmax(dim=1))
        return FeedForwardNetwork(nn_head, nn_tail)


class ClassificationTask(TaskFactory):
    def get_network(self, _, sample):
        assembled_network_factory = MyNetworkFactory(self.nn_head_factory)
        assembled_network = assembled_network_factory.create_network(
            task=None, input=sample)
        return assembled_network

    def create_task(
        self, data: IndexedDataBundle,
            env: Optional[TrainingEnvironment] = None) -> None:

        metrics = bt.MetricPool().add(MulticlassMetrics())
        self.instantiate_default_task(
            epoch_count=2,
            batch_size=100,
            mini_batch_size=50,
            mini_epoch_count=1,
            metric_pool=metrics
        )

        plain_context_builder = PlainContextBuilder(include_zero_offset=False,
                                                    left_to_right_contexts_proportion=1)

        plain_context = ContextualBinding(
            name='plain_context',
            context_length=3,
            network_type=ContextualNetworkType.Plain,
            hidden_size=[30],
            context_builder=plain_context_builder,
            extractor=CoreExtractor(join_column='another_word_id'),
            debug=False
        )
        self.nn_head_factory = plain_context.create_network_factory(
            task=None, input=None)  # TODO could be better?
        core_extractor = plain_context.create_extractor(task=None, bundle=data)
        self.setup_batcher(data, [core_extractor, get_multilabel_extractor()])
        self.setup_model(self.get_network)


def get_training_job() -> TrainingJob:
    task = ClassificationTask()
    task.info["dataset"] = dataset_name
    task.info["name"] = "gg_task"

    job = TrainingJob(
        tasks=[task],
        project_name=project_name,
        bucket=bucket
    )
    return job


job = get_training_job()

routine = SSHDockerJobRoutine(
    job=job,
    remote_host_address=None,
    remote_host_user=None,
    handler_factory=FakeContainerHandler.Factory(),
    options=DockerOptions(propagate_environmental_variables=["AWS_ACCESS_KEY_ID",
                                                             "AWS_SECRET_ACCESS_KEY"])
)
tag = 'v_' + datetime.datetime.now().time().strftime("%H_%M_%S")
dockerhub_repo = 'grammar_repo'  # name of your repo
dockerhub_login = 'sergio0x0'  # your login

local_img = 'gg_img'


# job.run()
b_path = Loc.bundles_path/'grammatical_gender/toy'
data = DataBundle.load(b_path)

task = ClassificationTask()
task.create_task(data)
temp_batch = task.task.generate_sample_batch(data,0)

2022-12-13 12:51:59.068430+00:00 INFO: Training starts. Info: {}
2022-12-13 12:51:59.069277+00:00 INFO: Ensuring/loading bundle. Bundle before:
{'syntax_stats': {'shape': (99977, 6), 'index_name': 'word_id'}, 'syntax_fixes': {'shape': (99977, 4), 'index_name': 'word_id'}, 'src': {'shape': (99977, 16), 'index_name': None}, 'pymorphy': {'shape': (99977, 16), 'index_name': 'word_id'}, 'slovnet': {'shape': (99977, 17), 'index_name': 'word_id'}, 'syntax_closure': {'shape': (260387, 4), 'index_name': 'entry_id'}, 'index': {'shape': (55787, 4), 'index_name': 'sample_id'}}
2022-12-13 12:51:59.071489+00:00 INFO: Bundle loaded
{'syntax_stats': {'shape': (99977, 6), 'index_name': 'word_id', 'columns': ['children', 'descendants', 'up_depth', 'down_depth', 'sentence_length', '...'], 'index': [0, 1, 2, 3, 4, '...']}, 'syntax_fixes': {'shape': (99977, 4), 'index_name': 'word_id', 'columns': ['syntax_parent_id', 'root', 'cycle_status', 'correct_root'], 'index': [0, 1, 2, 3, 4, '...']}, 'src': {'shape'

In [34]:
net = task.task.model_handler.network

In [35]:
net(temp_batch)

tensor([[0.5987, 0.4655, 0.4218, 0.5549],
        [0.5853, 0.4826, 0.3851, 0.5822],
        [0.5951, 0.4810, 0.3921, 0.5766],
        [0.5929, 0.4740, 0.4027, 0.5694],
        [0.6001, 0.4769, 0.3993, 0.5715],
        [0.6122, 0.4690, 0.3938, 0.5691],
        [0.6034, 0.4750, 0.4072, 0.5657],
        [0.6043, 0.4795, 0.4019, 0.5675],
        [0.6042, 0.4725, 0.4277, 0.5548],
        [0.6022, 0.4814, 0.4008, 0.5708],
        [0.5993, 0.4803, 0.3954, 0.5718],
        [0.5829, 0.4731, 0.4191, 0.5739],
        [0.6085, 0.4714, 0.4144, 0.5732],
        [0.5902, 0.4798, 0.4075, 0.5685],
        [0.5969, 0.4736, 0.4229, 0.5643],
        [0.6091, 0.4847, 0.4088, 0.5653],
        [0.5968, 0.4870, 0.4059, 0.5742],
        [0.6004, 0.4692, 0.4091, 0.5635],
        [0.5894, 0.4711, 0.4044, 0.5748],
        [0.5982, 0.4703, 0.4212, 0.5667],
        [0.5962, 0.4869, 0.4055, 0.5741],
        [0.5941, 0.4663, 0.4169, 0.5593],
        [0.5937, 0.4784, 0.4051, 0.5692],
        [0.5977, 0.4612, 0.4218, 0

AttributeError: 'FeedForwardNetwork' object has no attribute 'get_parameters'